<h1><center> Monthly Preipitation Totals using ERA5 <center></h1>
To get monthly precipitation totals, the precipitation data needs to be downloaded by the hour and then summed by the month. Downloading the monthly precipitation dataset through and API request will only provide you with the monthly average not the monthly sum. This code sums the daily precipitation in order to get monthly precipitation totals

<h2><center> API request for precipitation Data <center></h2>

In [20]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable':'total_precipitation',
        'year':[
            '2019'
        ],
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'day':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12',
            '13','14','15',
            '16','17','18',
            '19','20','21',
            '22','23','24',
            '25','26','27',
            '28','29','30',
            '31'
        ],
        'time':[
            '00:00','01:00','02:00',
            '03:00','04:00','05:00',
            '06:00','07:00','08:00',
            '09:00','10:00','11:00',
            '12:00','13:00','14:00',
            '15:00','16:00','17:00',
            '18:00','19:00','20:00',
            '21:00','22:00','23:00'
        ],
        'grid': "0.1 /0.1 ",
        'area':"57/-128/49/-120",
        'format':'netcdf'
    },
    'download.nc')

2019-08-30 10:06:16,557 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2019-08-30 10:06:17,851 INFO Request is queued


KeyboardInterrupt: 

<h3><center> Check to make sure the file name correspond to correct year <center></h3>
I created a filing system by year for the precipitation data so this code loops through the filing system to make sure all the years I have labeled the files with correspond to the year that is actually in the file. If the year does not match this code throughs an error that specifies which file is incorrect and the file year to rename it with

In [ ]:

for Y in np.arange(1979,2019,1):
    with Dataset('C:/Users/vore/Desktop/HydroMet_Project/Data/ERA5/Hourly_Precip_ERA5_land/%s_land.nc'%Y) as nc_file:
        t_units=nc_file.variables['time'].units
        t_cal=nc_file.variables['time'].calendar
        t_datetime=[]
        for x in np.arange(0,len(nc_file.variables['time'])):
            t=(float(np.ma.getdata(nc_file.variables['time'][x]))) 
            t_datetime.append(num2date(t,units=t_units,calendar=t_cal))
        t_month=[];t_year=[]
        for x in np.arange(0,len(t_datetime)):
            t_month.append(t_datetime[x].month)
            t_year.append(t_datetime[x].year)

        if t_year[0]!=Y:
            print("YEAR ERROR: %s.nc is actually %s"%(Y, t_year[0]))
     

<h3><center> Create Monthly Precipitation Sums <center></h3>
NOTE: 
The resulting dataframe does not contain the spatial information that is linked to the precipitation arrays that were created for each month. Thus, to use this data, make sure to load in the lat and long data into the script you are using! Also save this using pd.to_pickle to perseve the arrays in the precipitation column. If it is saved as a csv it will not perserve the array.


In [2]:
from netCDF4 import Dataset,num2date
import numpy as np
import pandas as pd

In [8]:
Month_Precip=pd.DataFrame({'Year':[],'Month':[],'TotalPrecip':[]})
Year_range=np.arange(1979,2019,1)
for Y in Year_range:
    with Dataset('C:/Users/vore/Desktop/HydroMet_Project/Data/ERA5/Hourly_Precip/%s.nc'%Y) as nc_file:
        #nc_file.variables
        precip=nc_file.variables['tp'][:,:,:]
        t_units=nc_file.variables['time'].units
        t_cal=nc_file.variables['time'].calendar
        
        t_datetime=[]
        for x in np.arange(0,len(nc_file.variables['time'])):
            t=(float(np.ma.getdata(nc_file.variables['time'][x]))) 
            t_datetime.append(num2date(t,units=t_units,calendar=t_cal))
            
        t_month=[];t_year=[]
        for x in np.arange(0,len(t_datetime)):
            t_month.append(t_datetime[x].month)
            t_year.append(t_datetime[x].year)  
        t=np.array([t_year,t_month]) 

        index_year=np.where(t[0]==Y)[0]
        for month in np.arange(1,13,1):
            precip_YEAR=nc_file.variables['tp'][min(index_year):max(index_year)+1,:,:]*1000
            index_month=np.where(t[1][min(index_year):max(index_year+1)]==month)[0]
            precip_MONTH=precip_YEAR[min(index_month):max(index_month)+1]
            precip_MONTH=np.array(precip_MONTH)
            p=np.sum(np.array(precip_MONTH),axis=0)        
            Month_Precip=Month_Precip.append({'Year':Y,'Month':month,'TotalPrecip':p},ignore_index=True)
# Month_Precip.to_pickle('C:/Users/vore/Desktop/HydroMet_Project/Data/ERA5/Precip_daily_totals/Montly_Precip.pkl')    

In [9]:
Month_Precip

,Year,Month,TotalPrecip
0,1979.0,1.0,"[[45.049678094565465, 42.67302355911041, 39.79..."
1,1979.0,2.0,"[[64.18048557462373, 65.75662615374442, 67.947..."
2,1979.0,3.0,"[[73.73190091683102, 72.55618895266053, 71.763..."
3,1979.0,4.0,"[[22.550842019921024, 22.7985138979207, 22.958..."
4,1979.0,5.0,"[[79.33017787089611, 78.4991125671715, 77.0568..."
5,1979.0,6.0,"[[74.17403727007232, 70.2102574013786, 66.3384..."
6,1979.0,7.0,"[[84.70635682764208, 89.99345629396403, 93.568..."
7,1979.0,8.0,"[[33.47191889119096, 36.365200143748716, 39.45..."
8,1979.0,9.0,"[[114.68855664503542, 108.49332695938129, 102...."
9,1979.0,10.0,"[[69.41214636000741, 64.22596932214275, 58.598..."
